In [1]:
!apt-get install -y python-rdkit librdkit1 rdkit-data
!pip install rdkit




E: Unable to locate package python-rdkit
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 49.0 MB/s eta 0:00:00


In [2]:
!pip install ogb

  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.7 MB/s eta 0:00:00
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=fee1c4f4e34eb1e636e8aea6fdcc481339c5d5fc85bf653c6d9aae2c9777d13a
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.6 MB/s eta 0:00:00


In [4]:
from ogb.lsc import PygPCQM4Mv2Dataset, PCQM4Mv2Evaluator
from ogb.graphproppred.mol_encoder import AtomEncoder,BondEncoder
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINEConv
from torch_geometric.nn import GINConv
from torch_geometric.nn.pool import global_add_pool

In [5]:
class GNN_graph(torch.nn.Module):
    def __init__(self, num_layers=5, emb_dim=100, drop_ratio=0.5):
        super().__init__()
        self.num_layers = num_layers
        self.drop_ratio = drop_ratio
        self.atom_encoder = AtomEncoder(emb_dim)
        #self.bond_encoder = BondEncoder(emb_dim)
        self.mlp = torch.nn.Sequential(torch.nn.Linear(emb_dim, emb_dim), torch.nn.BatchNorm1d(emb_dim), torch.nn.ReLU(), torch.nn.Linear(emb_dim, emb_dim))
        
        self.graph_pool = global_add_pool
        self.linear_pred = torch.nn.Linear(emb_dim, 1)
        
        if self.num_layers<2:
            raise ValueError("Number of layers must be more than 1")
            
        self.convs = torch.nn.ModuleList()
        self.norms = torch.nn.ModuleList()
        
        for i in range(num_layers):
            self.convs.append(GINConv(self.mlp))
            self.norms.append(torch.nn.BatchNorm1d(emb_dim))
            
    def forward(self, batched_data):
        x, edge_index, edge_attr, batch = batched_data.x, batched_data.edge_index, batched_data.edge_attr, batched_data.batch
        #edge_embedding = self.bond_encoder(edge_attr)
        h_list = [self.atom_encoder(x)]
        for layer in range(self.num_layers):

            h = self.convs[layer](h_list[layer], edge_index)
            h = self.norms[layer](h)

            if layer == self.num_layers - 1:
                #remove relu for the last layer
                h = F.dropout(h, self.drop_ratio, training = self.training)
            else:
                h = F.dropout(F.relu(h), self.drop_ratio, training = self.training)

            h_list.append(h)
            
            
        node_feat = h_list[-1]
        graph_feat = self.graph_pool(node_feat, batch)
        output = self.linear_pred(graph_feat)
        
        return output

In [6]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch_geometric.loader import DataLoader
import os
import time
import random
import numpy as np
from tqdm.auto import tqdm
from torch_geometric.datasets import PCQM4Mv2

In [7]:
train_dataset = PCQM4Mv2(root = 'dataset/', split = 'test')
val_dataset = PCQM4Mv2(root = 'dataset/', split = 'val')
print(train_dataset)
data = (train_dataset.get(12))
print((data.x).size())
print((data.edge_attr).size())
print((data.edge_index).size())

Extracting dataset/raw/pcqm4m-v2.zip
Processing...
100%|██████████| 3746620/3746620 [54:48<00:00, 1139.16it/s]
Done!


PCQM4Mv2(147037)
torch.Size([16, 9])
torch.Size([36, 3])
torch.Size([2, 36])


In [8]:
reg_criterion = torch.nn.L1Loss()

In [9]:
!pwd

/kaggle/working


In [10]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
random.seed(42)

In [11]:
batch_size = 128
epochs = 20
drop_ratio = 0.5
device = torch.device("cuda")  if torch.cuda.is_available() else torch.device("cpu")

In [12]:
subset_ratio = 0.15
train_idx = torch.randperm(len(train_dataset))[:int(subset_ratio*len(train_dataset))]
train_loader = DataLoader(train_dataset[train_idx], batch_size=batch_size, shuffle=True, num_workers = 0)
    
valid_idx = torch.randperm(len(val_dataset))[:int(0.03*len(val_dataset))]
valid_loader = DataLoader(val_dataset[valid_idx], batch_size=batch_size, shuffle=True, num_workers = 0)
    
evaluator = PCQM4Mv2Evaluator()

In [13]:
print(len(train_idx))
print(len(valid_idx))

22055
2206


In [14]:
checkpoint_dir = '/kaggle/working/checkpoints'
os.makedirs(checkpoint_dir)
    
model = GNN_graph(num_layers=5, emb_dim=200, drop_ratio=0.5).to(device)
    
num_params = sum(p.numel() for p in model.parameters())
print(f'#Params: {num_params}')
    
optimizer = optim.SGD(model.parameters(), lr=0.1)

#Params: 117801


In [15]:
def train(model, device, loader, optimizer):
    model.train()
    
    loss_accum = 0
    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        pred = model(batch).view(-1,)
        optimizer.zero_grad()
        loss = reg_criterion(pred, batch.y)
        loss.backward()
        optimizer.step()

        loss_accum += loss.detach().cpu().item()
        
        torch.cuda.empty_cache()

    return loss_accum / (step + 1)

In [16]:
def eval(model, device, loader, evaluator):
    model.eval()
    
    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        with torch.no_grad():
            pred = model(batch).view(-1,)

        y_true.append(batch.y.view(pred.shape).detach().cpu())
        y_pred.append(pred.detach().cpu())
        
        torch.cuda.empty_cache()

    y_true = torch.cat(y_true, dim = 0)
    y_pred = torch.cat(y_pred, dim = 0)

    input_dict = {"y_true": y_true, "y_pred": y_pred}

    return evaluator.eval(input_dict)["mae"]

In [17]:
def test(model, device, loader):
    model.eval()
    
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        with torch.no_grad():
            pred = model(batch).view(-1,)

        y_pred.append(pred.detach().cpu())

    y_pred = torch.cat(y_pred, dim = 0)

    return y_pred

In [18]:
best_valid_mae = 1000
    
scheduler = StepLR(optimizer, step_size = 4, gamma = 0.5)
    
for epoch in range(1, epochs+1):
    print("=====Epoch {}".format(epoch))
    print('Training...')
    train_mae = train(model, device, train_loader, optimizer)

    print('Evaluating...')
    valid_mae = eval(model, device, valid_loader, evaluator)

    print({'Train': train_mae, 'Validation': valid_mae})
        
    if valid_mae < best_valid_mae:
        best_valid_mae = valid_mae
        if checkpoint_dir != '':
            print('Saving checkpoint...')
            checkpoint = {'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'scheduler_state_dict': scheduler.state_dict(), 'best_val_mae': best_valid_mae, 'num_params': num_params}
            torch.save(checkpoint, os.path.join(checkpoint_dir, 'checkpoint.pt'))
                
    scheduler.step()
        
    print(f'Best valid MAE so far: {best_valid_mae}')

=====Epoch 1
Training...


Iteration:   0%|          | 0/173 [00:00<?, ?it/s]

Evaluating...


Iteration:   0%|          | 0/18 [00:00<?, ?it/s]

{'Train': 19.56495898583032, 'Validation': 1.21367609500885}
Saving checkpoint...
Best valid MAE so far: 1.21367609500885
=====Epoch 2
Training...


...

{'Train': 0.5206244637855905, 'Validation': 1.383447289466858}
Best valid MAE so far: 1.0529060363769531
=====Epoch 20
Training...


Iteration:   0%|          | 0/173 [00:00<?, ?it/s]

Evaluating...


Iteration:   0%|          | 0/18 [00:00<?, ?it/s]

{'Train': 0.5226879250796544, 'Validation': 1.3065210580825806}
Best valid MAE so far: 1.0529060363769531
